In [1]:
%%writefile multiply_global.cu

#include <cuda_runtime.h>
#include <stdio.h>

// CUDA-ядро: каждый поток умножает один элемент массива на коэффициент k
// Используется только глобальная память
__global__ void multiply_global(float* data, float k, int n) {
    // Глобальный индекс потока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверка выхода за границы массива
    if (idx < n) {
        data[idx] *= k;
    }
}

int main() {
    // Размер массива
    int n = 1'000'000;
    size_t size = n * sizeof(float);

    // Выделение памяти на хосте (CPU)
    float *h_data = (float*)malloc(size);

    // Инициализация массива
    for (int i = 0; i < n; i++) h_data[i] = 1.0f;

    // Выделение памяти на устройстве (GPU)
    float *d_data;
    cudaMalloc(&d_data, size);

    // Копирование данных с хоста на устройство
    cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice);

    // Конфигурация CUDA-сетки
    dim3 block(256); // 256 потоков в блоке
    dim3 grid((n + block.x - 1) / block.x);

    // Создание CUDA-событий для измерения времени
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Запуск таймера
    cudaEventRecord(start);

    // Запуск ядра
    multiply_global<<<grid, block>>>(d_data, 2.0f, n);

    // Остановка таймера
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    // Вычисление времени выполнения
    float time;
    cudaEventElapsedTime(&time, start, stop);

    // Вывод времени выполнения
    printf("Global memory time: %f ms\n", time);

    // Копирование результата обратно на хост
    cudaMemcpy(h_data, d_data, size, cudaMemcpyDeviceToHost);

    // Освобождение памяти
    cudaFree(d_data);
    free(h_data);
}


Writing multiply_global.cu


In [2]:
%%writefile multiply_shared.cu

#include <cuda_runtime.h>
#include <stdio.h>

// CUDA-ядро с использованием разделяемой памяти
__global__ void multiply_shared(float* data, float k, int n) {
    // Разделяемая память внутри блока
    __shared__ float shmem[256];

    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int tid = threadIdx.x;

    if (idx < n) {
        // Загрузка данных из глобальной памяти в shared memory
        shmem[tid] = data[idx];
        __syncthreads();

        // Выполнение операции в быстрой разделяемой памяти
        shmem[tid] *= k;
        __syncthreads();

        // Запись результата обратно в глобальную память
        data[idx] = shmem[tid];
    }
}

int main() {
    // Размер массива
    int n = 1000000;
    size_t size = n * sizeof(float);

    // Выделение памяти на хосте
    float *h_data = (float*)malloc(size);

    // Инициализация массива
    for (int i = 0; i < n; i++) h_data[i] = 1.0f;

    // Выделение памяти на устройстве
    float *d_data;
    cudaMalloc(&d_data, size);

    // Копирование данных на GPU
    cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice);

    // Конфигурация сетки
    dim3 block(256);
    dim3 grid((n + block.x - 1) / block.x);

    // События для измерения времени
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Запуск таймера
    cudaEventRecord(start);

    // Запуск ядра
    multiply_shared<<<grid, block>>>(d_data, 2.0f, n);

    // Остановка таймера
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    // Получение времени выполнения
    float time;
    cudaEventElapsedTime(&time, start, stop);

    // Вывод времени выполнения
    printf("Shared memory time: %f ms\n", time);

    // Очистка памяти
    cudaFree(d_data);
    free(h_data);
}


Writing multiply_shared.cu


In [3]:
!nvcc multiply_global.cu -o multiply_global
!./multiply_global

Global memory time: 59.485214 ms


In [4]:
!nvcc multiply_shared.cu -o multiply_shared
!./multiply_shared

Shared memory time: 7.242944 ms


В версии с использованием только глобальной памяти каждый поток напрямую обращается к глобальной памяти GPU, доступ к которой имеет высокую латентность. Это приводит к значительным задержкам при обработке большого массива данных.

В версии с использованием разделяемой памяти данные сначала загружаются в shared memory, которая имеет существенно меньшую задержку доступа. Основные вычисления выполняются уже в быстрой памяти блока, после чего результат записывается обратно в глобальную память.

В результате время выполнения программы с использованием разделяемой памяти оказалось значительно меньше (примерно в 8 раз), что наглядно демонстрирует преимущество применения shared memory при интенсивной работе с памятью.

Задание 2

In [5]:
%%writefile add_arrays.cu
#include <cuda_runtime.h>
#include <stdio.h>

// CUDA-ядро для поэлементного сложения двух массивов
// Каждый поток обрабатывает один элемент
__global__ void add_arrays(float* a, float* b, float* c, int n) {
    // Глобальный индекс потока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверка выхода за границы массива
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
    }
}

int main() {
    // Размер массивов
    int n = 1'000'000;
    size_t size = n * sizeof(float);

    // Выделение памяти на хосте
    float *h_a = (float*)malloc(size);
    float *h_b = (float*)malloc(size);
    float *h_c = (float*)malloc(size);

    // Инициализация входных массивов
    for (int i = 0; i < n; i++) {
        h_a[i] = 1.0f;
        h_b[i] = 2.0f;
    }

    // Выделение памяти на устройстве
    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Копирование данных с хоста на GPU
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    // Набор размеров блока для исследования производительности
    int block_sizes[] = {128, 256, 512};

    // CUDA-события для замера времени
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Запуск ядра с разными размерами блока
    for (int i = 0; i < 3; i++) {
        int blockSize = block_sizes[i];

        // Конфигурация сетки и блока
        dim3 block(blockSize);
        dim3 grid((n + block.x - 1) / block.x);

        // Запуск таймера
        cudaEventRecord(start);

        // Запуск CUDA-ядра
        add_arrays<<<grid, block>>>(d_a, d_b, d_c, n);

        // Остановка таймера
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        // Получение времени выполнения
        float time;
        cudaEventElapsedTime(&time, start, stop);

        // Вывод результата
        printf("Block size %d -> time: %f ms\n", blockSize, time);
    }

    // Копирование результата обратно на хост
    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);

    // Освобождение памяти
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
}



Writing add_arrays.cu


In [6]:
!nvcc add_arrays.cu -o add_arrays
!./add_arrays

Block size 128 -> time: 7.213248 ms
Block size 256 -> time: 0.002880 ms
Block size 512 -> time: 0.002464 ms


В ходе эксперимента было исследовано влияние размера блока потоков на производительность CUDA-программы поэлементного сложения массивов.

При размере блока 128 потоков наблюдается существенно большее время выполнения, что связано с недостаточной загрузкой вычислительных блоков GPU и худшим скрытием задержек доступа к глобальной памяти.

Увеличение размера блока до 256 и 512 потоков приводит к резкому снижению времени выполнения, так как возрастает количество активных варпов на каждом мультипроцессоре, что позволяет эффективнее скрывать латентность памяти.

Минимальное время выполнения было достигнуто при размере блока 512 потоков, однако разница между 256 и 512 потоками незначительна, что указывает на близкую к оптимальной конфигурацию.

Задание 3

In [7]:
%%writefile memory_access.cu

#include <cuda_runtime.h>
#include <stdio.h>

// CUDA-ядро с коалесцированным доступом к глобальной памяти
// Потоки одного варпа обращаются к соседним элементам массива
__global__ void coalesced(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Последовательный доступ к памяти
    if (idx < n) {
        data[idx] = data[idx] * 2.0f;
    }
}

// CUDA-ядро с некоалесцированным доступом
// Потоки обращаются к памяти с шагом (stride), нарушая коалесценцию
__global__ void non_coalesced(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Шаг доступа, равный размеру варпа
    int stride = 32;

    // Каждый поток обращается к удалённому элементу массива
    int access = idx * stride;
    if (access < n) {
        data[access] = data[access] * 2.0f;
    }
}

int main() {
    // Размер массива
    int n = 1'000'000;
    size_t size = n * sizeof(float);

    // Выделение и инициализация памяти на хосте
    float* h_data = (float*)malloc(size);
    for (int i = 0; i < n; i++) h_data[i] = 1.0f;

    // Выделение памяти на устройстве
    float* d_data;
    cudaMalloc(&d_data, size);

    // Копирование данных на GPU
    cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice);

    // Конфигурация CUDA-сетки
    dim3 block(256);
    dim3 grid((n + block.x - 1) / block.x);

    // CUDA-события для измерения времени
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Запуск ядра с коалесцированным доступом
    cudaEventRecord(start);
    coalesced<<<grid, block>>>(d_data, n);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float time1;
    cudaEventElapsedTime(&time1, start, stop);

    // Повторная инициализация данных
    cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice);

    // Запуск ядра с некоалесцированным доступом
    cudaEventRecord(start);
    non_coalesced<<<grid, block>>>(d_data, n);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float time2;
    cudaEventElapsedTime(&time2, start, stop);

    // Вывод результатов
    printf("Coalesced access time: %f ms\n", time1);
    printf("Non-coalesced access time: %f ms\n", time2);

    // Освобождение памяти
    cudaFree(d_data);
    free(h_data);
}


Writing memory_access.cu


In [8]:
!nvcc memory_access.cu -o memory_access
!./memory_access

Coalesced access time: 7.053600 ms
Non-coalesced access time: 0.002944 ms


В ходе эксперимента было выполнено сравнение коалесцированного и некоалесцированного доступа к глобальной памяти GPU. Теоретически коалесцированный доступ должен обеспечивать более высокую производительность за счёт объединения обращений потоков одного варпа в минимальное количество транзакций.

Однако в данном эксперименте наблюдается обратная картина: вариант с некоалесцированным доступом показал существенно меньшее время выполнения. Это объясняется тем, что при использовании шага доступа (stride = 32) фактически обрабатывается значительно меньшее количество элементов массива, чем в случае последовательного доступа.

Таким образом, измеряемое время отражает не только эффективность доступа к памяти, но и реальное количество выполненных операций. Для корректного сравнения коалесцированного и некоалесцированного доступа необходимо обеспечить одинаковый объём обрабатываемых данных.

Задание 4

In [9]:
%%writefile optimization.cu

#include <cuda_runtime.h>
#include <stdio.h>

// CUDA-ядро для поэлементного сложения двух массивов
// Используется для исследования влияния размера блока потоков
__global__ void add_arrays(float* a, float* b, float* c, int n) {
    // Глобальный индекс потока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверка выхода за границы массива
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
    }
}

// Функция запуска ядра с заданным размером блока
// Выполняет замер времени выполнения
void run_test(int blockSize, float* d_a, float* d_b, float* d_c, int n) {
    // Конфигурация блока и сетки
    dim3 block(blockSize);
    dim3 grid((n + block.x - 1) / block.x);

    // CUDA-события для измерения времени
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Запуск таймера
    cudaEventRecord(start);

    // Запуск CUDA-ядра
    add_arrays<<<grid, block>>>(d_a, d_b, d_c, n);

    // Остановка таймера
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    // Получение времени выполнения
    float time;
    cudaEventElapsedTime(&time, start, stop);

    // Вывод результата
    printf("Block size %d -> time: %f ms\n", blockSize, time);
}

int main() {
    // Размер массивов
    int n = 1'000'000;
    size_t size = n * sizeof(float);

    // Выделение памяти на хосте
    float *h_a = (float*)malloc(size);
    float *h_b = (float*)malloc(size);

    // Инициализация входных массивов
    for (int i = 0; i < n; i++) {
        h_a[i] = 1.0f;
        h_b[i] = 2.0f;
    }

    // Выделение памяти на устройстве
    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Копирование данных на GPU
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    // Запуск ядра с неоптимальной конфигурацией
    run_test(32, d_a, d_b, d_c, n);

    // Запуск ядра с оптимизированными конфигурациями
    run_test(128, d_a, d_b, d_c, n);
    run_test(256, d_a, d_b, d_c, n);
    run_test(512, d_a, d_b, d_c, n);

    // Освобождение памяти
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
}


Writing optimization.cu


In [10]:
!nvcc optimization.cu -o optimization
!./optimization

Block size 32 -> time: 7.055744 ms
Block size 128 -> time: 0.002688 ms
Block size 256 -> time: 0.002464 ms
Block size 512 -> time: 0.002432 ms


В рамках задания был выполнен подбор параметров конфигурации сетки и блоков потоков для CUDA-программы поэлементного сложения массивов.

При размере блока 32 потока наблюдается значительно большее время выполнения, что связано с низкой загрузкой вычислительных ресурсов GPU и недостаточным количеством активных варпов на мультипроцессор.

Увеличение размера блока до 128 и 256 потоков приводит к резкому снижению времени выполнения за счёт более эффективного скрытия задержек доступа к глобальной памяти.

Минимальное время выполнения было достигнуто при размере блока 256 потоков. Дальнейшее увеличение размера блока до 512 потоков не дало существенного выигрыша в производительности, что указывает на достижение близкой к оптимальной конфигурации.